<a href="https://colab.research.google.com/github/shizoda/education/blob/main/agent/LangChain(2)_PromptTemplate_LCEL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📘 第2回：プロンプトの構造化と LCEL パイプライン

第1回では、LLM へのリクエストが最終的に「JSON形式のテキスト」になることを確認しました。
しかし、複雑なアプリケーションを作る際、毎回 JSON や辞書のリストを手動で組み立てるのは効率が悪く、保守性も低いです。

第2回では、以下の技術を学び、LLM アプリケーションの「標準的な構築パターン」を習得します。

1.  **Prompt Template:** プロンプトを「単なる文字列」ではなく、変数を持つ「構造化されたオブジェクト」として定義する。
2.  **LCEL (LangChain Expression Language):** UNIXのパイプ (`|`) のように処理を連結する構文。
3.  **Output Parser:** LLM の出力オブジェクトから、必要なデータ（文字列など）を自動で抽出する。

### 🎯 今回の学習目標
* Python 標準の f-string と LangChain の Template の違いをコードレベルで理解する。
* `|` 演算子を使って、コンポーネント（Prompt, Model, Parser）を接続する。
* **Runnable Protocol**（データフローの統一規格）を理解する。

In [ ]:
# 📦 ライブラリの準備
!pip install -qU langchain-openai langchain-core termcolor

# 🔑 API キーの準備（Colab シークレットから読み込み）
import os
from google.colab import userdata

try:
    if "OPENROUTER_API_KEY" not in os.environ:
        os.environ["OPENROUTER_API_KEY"] = userdata.get("OPENROUTER_API_KEY")
    print("✅ API Key loaded.")
except Exception as e:
    print("❌ Error: Please set OPENROUTER_API_KEY in secrets.")

### 1️⃣ プロンプト構築：Python 標準 vs LangChain

まずは、動的にプロンプトを作る場面を考えます。
例えば、「役割（Role）」と「話題（Topic）」を変数として受け取り、メッセージリストを作る関数です。

#### 【A】🐍 Python 標準機能 (f-string) での直書き

LangChain の `PromptTemplate` を理解するために、まずは Python 標準の f-string を使って同様の処理を記述します。
ここでは関数化を行わず、**「辞書を要素に持つリスト」** から **「プロンプト（文字列）のリスト」** を生成する過程を可視化します。

以下のコードを実行し、データの流れを確認してください。

In [ ]:
from termcolor import cprint

# 1. テンプレート（ひな形）の定義
template = "私の名前は{name}です。職業は{occupation}です。"

# 2. データの準備（辞書のリスト）
data_list = [
    {"name": "田中", "occupation": "エンジニア"},
    {"name": "佐藤", "occupation": "データサイエンティスト"},
    {"name": "鈴木", "occupation": "プロジェクトマネージャー"},
]

# 3. f-string による置換処理（直書き）
print("--- プロンプト ---")
generated_prompts = []

for item in data_list:
    # 辞書のキーを指定して f-string に埋め込む
    # **item でアンパックする方法もありますが、ここでは明示的にキーを指定しています
    prompt = f"私の名前は{item['name']}です。職業は{item['occupation']}です。"

    generated_prompts.append(prompt)

    # 確認用出力
    cprint(f"Input: {item} => Output: {prompt}", "cyan")

print("\n--- 結果のリスト ---")
print(generated_prompts)

### 🦜 【B】 LangChain の PromptTemplate での実装

Python 標準機能では、`for` ループや `format` 関数を自分で記述する必要がありました。
LangChain を使用すると、これらの処理をより構造化し、簡潔に記述できます。

特に大量のデータを処理する場合、LangChain の **`batch`** 機能を使うことで、ループ処理を書かずに「辞書のリスト」を一括で「プロンプトのリスト」に変換できる点が大きなメリットです。

#### 1. ChatPromptTemplate の基本
ここでは `ChatPromptTemplate` を使用します。これは、単なる文字列置換だけでなく、LLM に渡すための「メッセージ構造（System, Human など）」を管理するクラスです。

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

# 1. テンプレートの定義
# from_template で文字列から簡易に作成できます
prompt_template = ChatPromptTemplate.from_template(
    "私の名前は {name} です。職業は {occupation} です。"
)

# 中身を確認
print(f"Input Variables: {prompt_template.input_variables}")

#### 2. batch による一括処理
先ほど Python で `for` 文を使って書いた処理は、LangChain では **`.batch()`** メソッド 1 つで完結します。

* **Python 手動実装**: リストをループ → 1つずつ置換 → リストに追加
* **LangChain**: `prompt_template.batch(辞書リスト)`

データ（辞書のリスト）をそのまま渡すだけで、並列処理を考慮した効率的な変換が行われます。

In [ ]:
# 2. データの準備（先ほどと同じ辞書のリスト）
data_list = [
    {"name": "田中", "occupation": "エンジニア"},
    {"name": "佐藤", "occupation": "データサイエンティスト"},
    {"name": "鈴木", "occupation": "プロジェクトマネージャー"},
]

# 3. 一括変換 (batch実行)
# for文は不要です。リストをそのまま渡します。
generated_messages = prompt_template.batch(data_list)

# 結果の確認
print("--- .batch() の結果 ---")
for i, msg in enumerate(generated_messages):
    # ChatPromptTemplate の出力は "ChatPromptValue" という特殊な型ですが
    # ここでは可読性のため文字列化して表示します
    content = msg.to_string()
    cprint(f"[{i}] {content}", "cyan")

### 📝 解説: 何が便利になったのか？

比較すると以下の点が効率化されています。

1.  **ループ処理の隠蔽**: `for` 文を書く必要がなく、コードの記述量が減り、可読性が向上しました。
2.  **構造化データの扱い**: 単なる文字列置換ではなく、チャットモデル（ChatGPT 等）が理解しやすいメッセージオブジェクト（`HumanMessage` 等）として自動的に生成されています。

次節では、このプロンプトを使って実際に LLM を呼び出してみましょう。

### 2️⃣ LCEL: パイプラインによる処理の連結

ここからが LangChain の核心部分です。
LangChain では、`Prompt` や `Model` などのコンポーネントを **`|` (パイプ) 演算子** で繋ぐことができます。
これを **LCEL (LangChain Expression Language)** と呼びます。

数式のように書くことができます：
$$Chain = Prompt \mid Model$$

この書き方をすると、**「Prompt の出力」が自動的に「Model の入力」として渡されます。**

In [ ]:
from langchain_openai import ChatOpenAI
import os

# 1. モデルの準備
llm = ChatOpenAI(
    model="deepseek/deepseek-chat-v3-0324",
    api_key=os.environ["OPENROUTER_API_KEY"],
    base_url="https://openrouter.ai/api/v1",
    temperature=0.7
)

# 2. プロンプトテンプレートの準備
template = ChatPromptTemplate.from_messages([
    ("system", "あなたはプロのライターです。ユーザーの情報を元に、SNS用の魅力的なプロフィール文を考案してください。"),
    ("human", "私の名前は {name} です。職業は {occupation} です。絵文字を使って親しみやすく書いてください。"),
])


# ------------------------------------------------------------------
# 【課題】 チェーンの実行
# ------------------------------------------------------------------
# 作成した `chain` に対して .invoke() を実行してください。
# 引数には、前のステップと同様の辞書（例: {"name": "...", "occupation": "..."}）を1つ渡します。

# ↓ ここにコードを記述し、実行結果(content)を表示してください

# 3. チェーンの構築
chain = ■■■

# 実行に必要なデータを辞書で用意
input_data = {"name": "田中", "occupation": "エンジニア"}

print(f"Running chain with: {input_data} ...\n")

# チェーンの実行
response = chain.invoke(input_data)

# 結果の表示
print("--- AIの生成結果 ---")
print(response.■■■)

### 3️⃣ OutputParser: 出力の加工

`chain.invoke` の結果は `AIMessage` オブジェクトでした。しかし、多くのアプリケーションでは、メタデータを含まない「純粋なテキスト（文字列）」だけが必要です。

そこで、チェーンの最後に **`StrOutputParser`** を接続します。

$$Chain = Prompt \mid Model \mid Parser$$

これにより、入力から最終的な文字列出力までを一本のパイプラインとして定義できます。

In [ ]:
from langchain_core.output_parsers import StrOutputParser

# Parser の初期化
parser = StrOutputParser()

# ------------------------------------------------------------------
# 【演習 4】 完全なチェーンの構築
# ------------------------------------------------------------------
# 1. template
# 2. llm
# 3. parser
# この3つを `|` で繋いで、新しい `text_chain` を作成してください。
# その後、invoke を実行し、戻り値が `AIMessage` ではなく `str` (文字列) になっていることを確認してください。

# ↓ ここにコードを記述してください

In [ ]:
# 1. チェーンの再構築
# template (入力作成) -> llm (生成) -> parser (出力整形)
text_chain = ■■■

# 2. 実行 (前回と同じデータを使用)
input_data = {"name": "田中", "occupation": "エンジニア"}

print(f"Running text_chain with: {input_data} ...\n")
result = ■■■

# 3. 結果の確認
# AIMessage ではなく、純粋な str (文字列) が返ってきていることを確認します
print(f"--- Result Type: {type(result)} ---")
print(result)

### 🧠 なぜ `|` で繋がるのか？ (Runnable Protocol)

Python の標準機能に `|` でクラス同士を繋ぐ機能はありません。LangChain がこれを可能にしているのは、すべてのコンポーネント（Prompt, Model, Parserなど）が **`Runnable`** という共通の親クラスを継承しているからです。

`Runnable` プロトコルに従うオブジェクトは、以下のルールを守っています。

1.  **入力と出力の型が決まっている:**
    * Prompt: `辞書` → `PromptValue`
    * Model: `PromptValue` (or list) → `ChatMessage`
    * Parser: `ChatMessage` → `文字列`
2.  **`invoke` メソッドを持つ:** すべて共通の方法で呼び出せる。
3.  **`|` 演算子 (`__or__`) を実装している:** 前の出力を次の入力に渡す処理（Linux のパイプと同様）を行う。

この統一規格があるため、部品（Model や Prompt）を自由に入れ替えてもコードが壊れにくくなります。

---

### 次回の予告
今回は「文字列」を入力し「文字列」を出力するチェーンを作りました。
しかし、実務では **「JSON データを出力させて、プログラムで計算に使いたい」** という場面が多々あります。
次回は、LLM の出力を構造化データ（JSON）として取り出す **Output Parser の応用** と、複数のチェーンを並列に動かす方法を学びます。